In [1]:
import os
import rasterio
import geopandas as gpd
import numpy as np
from tqdm import tqdm
from shapely.geometry import box, mapping
from rasterio.merge import merge
from rasterio.features import geometry_mask

import geopandas as gpd
import numpy as np
import rasterio
from rasterio.mask import mask
from tqdm import tqdm
import os

## **Annual Cropmap Generation**

### **Overview**
This notebook processes annual cropmap data to generate masked images of cultivated areas within the Ukrainian boundary. The workflow involves the following steps:

1. **Select valid TIFF files**: Identify TIFF files that overlap with the Ukrainian boundary.
2. **Merge valid TIFF files for a specific year**: Extract the band corresponding to the specified year and merge the overlapping files.
3. **Mask the merged image with the Ukrainian boundary**: Apply the Ukrainian polygon as a mask to retain only the cultivated areas within the boundary.
4. **Save the result for each year**: Save the masked image as a new TIFF file for each specified year.

### **Data Sources**
- **Cropmap Data**:  
  The original cropmap data used in this notebook is **GLC_FCS30D**, the first global 30-m land-cover dynamic monitoring product with a fine classification system, covering the period from **1985 to 2022**.  
  Citation:  
  *Zhang, X., Zhao, T., Xu, H., Liu, W., Wang, J., Chen, X., & Liu, L. (2023). GLC_FCS30D: The first global 30-m land-cover dynamic monitoring product with a fine classification system from 1985 to 2022 using dense time-series Landsat imagery and continuous change-detection method. Earth System Science Data Discussions, 2023, 1-32.*  
  Link: [Zenodo: GLC_FCS30D](https://zenodo.org/records/8239305)

- **Ukrainian Boundary**:  
  The Ukrainian administrative boundary used for masking was obtained from **FAO GAUL: Global Administrative Unit Layers 2015**, which provides global administrative boundaries as of 2015.  
  This dataset was downloaded using **Google Earth Engine**, which provides access to FAO GAUL boundaries at multiple levels.  
  Link: [Google Earth Engine: FAO GAUL 2015 Level 0](https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0)

In [ ]:
# Path settings
cropmap_dir = '/home/sehoon/Desktop/fallow_land/cropmap/original/'
ua_json_path = '/home/sehoon/Desktop/fallow_land/Ukraine_Polygon_GAUL2015.geojson'
output_dir = '/mnt/raid5/1114/cropmap/'

# Load Ukrainian polygon
ua_polygons = gpd.read_file(ua_json_path)

# Check if the coordinate system is EPSG:4326 and convert if necessary
if ua_polygons.crs.to_epsg() != 4326:
    ua_polygons = ua_polygons.to_crs(epsg=4326)

# Search for TIFF files containing "Annual" in the specified cropmap directory
tif_files = []
for root, dirs, files in os.walk(cropmap_dir):
    for file in files:
        if file.endswith('.tif') and 'Annual' in file:
            tif_files.append(os.path.join(root, file))

# Function to check whether the boundary of the TIFF file overlaps with the Ukrainian polygon
def check_overlap(tif_path, ua_polygons):
    with rasterio.open(tif_path) as src:
        tif_bounds = box(*src.bounds)  # Create a bounding box for the TIFF file
        # Check if the TIFF file overlaps with the Ukrainian polygon
        if ua_polygons.intersects(tif_bounds).any():
            return True
    return False

# 1. Filter only TIFF files that overlap with the Ukrainian polygon (with tqdm visualization)
valid_tif_files = [tif for tif in tqdm(tif_files, desc="Checking overlap with polygons") if check_overlap(tif, ua_polygons)]

# 2. Merge valid TIFF files
def merge_tif_files(tif_files, band_index):
    src_files_to_mosaic = []
    
    # Open valid TIFF files and read the specified band, then add them to the list for merging
    for tif in tif_files:
        src = rasterio.open(tif)
        src_files_to_mosaic.append(src)  # Add the DatasetReader object itself
    
    # Merge TIFF files (processing only the specified band)
    mosaic, out_trans = merge(src_files_to_mosaic, indexes=[band_index])  # Merge the specified band
    
    # Get metadata for the merged data
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "count": 1,  # Since only the specified band was merged, set the number of bands to 1
        "dtype": mosaic.dtype
    })
    
    # Close the open files
    for src in src_files_to_mosaic:
        src.close()
    
    return mosaic, out_meta

# 3. Overlay the merged image with the Ukrainian polygon and mask only the overlapping area
def mask_with_polygon(mosaic, out_meta, polygons, output_path):
    # Convert polygons to GeoJSON format
    shapes = [mapping(geom) for geom in polygons.geometry]

    # Use geometry_mask to create a mask for the area to be retained
    mask_arr = geometry_mask(shapes, transform=out_meta['transform'],
                             invert=True, out_shape=mosaic.shape[1:])  # Only pass (height, width)

    # Generate the masked result (retain the unmasked area and set the masked area to 0)
    masked_mosaic = np.where(mask_arr, mosaic[0], 0)  # Use mosaic[0] to process only the first band

    # Save the masked result
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(masked_mosaic, 1)  # Save only the first band

# Main execution code
if __name__ == "__main__":
    # 2. Merge valid TIFF files and process data for 2013~2022
    if len(valid_tif_files) > 0:
        for year in tqdm(range(2013, 2024), desc="Processing years"):  # Apply tqdm
            band_index = year - 1999  # Since 2000 is the first band, 2013 corresponds to the 14th band
            output_file = os.path.join(output_dir, f"masked_cropmap_{year}.tif")  # Generate a different filename for each year

            # Merge valid TIFF files (processing the band corresponding to the specified year)
            mosaic, out_meta = merge_tif_files(valid_tif_files, band_index)
        
            # Mask the merged result with the Ukrainian polygon and save only the overlapping area
            mask_with_polygon(mosaic, out_meta, ua_polygons, output_file)
        
            print(f"Masked image for {year} saved to: {output_file}")
    else:
        print("No valid TIFF files overlap with the Ukrainian polygon.")

## **Cropmap Processing by Region**

### **Overview**
This script processes annual cropmap data to generate binary maps of cultivated areas for each region specified in a GeoJSON file. The primary goal is to identify cultivated areas based on user-defined pixel values and a threshold indicating the minimum number of years for which an area must be classified as cultivated.

In [ ]:
# Pixel values considered as cropland
DEFAULT_CROPLAND_VALUES = [10, 11, 12, 20]  # Rainfed, Herbaceous, Orchard, Irrigated

def get_polygons_from_geojson(geojson_file):
    """
    Function to extract polygons and names of each region from a GeoJSON file.
    """
    # Read the GeoJSON file
    gdf = gpd.read_file(geojson_file)

    # Extract polygons and names
    regions = []
    for _, row in gdf.iterrows():
        if row['geometry'].is_empty:
            continue
        regions.append((row['ADM1_NAME'], row['geometry']))  # Assuming 'ADM1_NAME' column is used as region name

    if not regions:
        raise ValueError("No valid polygons found in the GeoJSON file.")
    
    return regions

def crop_raster_with_polygon(src, polygon):
    """
    Function to crop a given polygon area from a TIFF file.
    """
    out_image, out_transform = mask(src, [polygon], crop=True)
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,  # Apply the transformed coordinates of the cropped area
    })
    return out_image, out_meta

def process_cropmap(file_paths, polygon, cropland_values=None, threshold=5, output_file=None):
    """
    Function to process TIFF files, calculate cropland areas, and save the results.
    """
    # Use default cropland values if none are provided
    if cropland_values is None:
        cropland_values = DEFAULT_CROPLAND_VALUES

    # Set metadata based on the first TIFF file
    with rasterio.open(file_paths[0]) as src:
        meta = src.meta

    total_pixels = []

    print(f"Loading TIFF files and cropping with polygons... (Number of files: {len(file_paths)})")
    for file_path in tqdm(file_paths, desc="Processing files"):
        with rasterio.open(file_path) as src:
            # Crop the TIFF file using the polygon
            out_image, out_meta = crop_raster_with_polygon(src, polygon)
            total_pixels.append(out_image[0])  # Single band (use only the first band)

    total_pixels = np.stack(total_pixels)

    # Determine whether each pixel is cropland or not
    print("Calculating cropland areas using vectorized operations...")

    # Create a mask (True/False) for pixels classified as cropland for each year
    cropland_mask = np.isin(total_pixels, cropland_values)

    # Only consider pixels as cropland if they are classified as cropland for at least the specified threshold of years
    result_map = np.sum(cropland_mask, axis=0) >= threshold

    # Assign 255 to cropland pixels and 0 to non-cropland pixels
    result_map = np.where(result_map, 255, 0).astype(np.uint8)

    # Save the result
    with rasterio.open(output_file, 'w', **out_meta) as dst:
        dst.write(result_map, 1)  # Save as a single band

    print(f"The new single-band cropland map TIFF file has been saved at {output_file}.")
    return output_file

def generate_cropmaps_by_region(geojson_file, file_paths, output_base_dir, cropland_values=None, threshold=5):
    """
    Function to generate cropland maps for all regions defined in a GeoJSON file.
    :param geojson_file: Path to the GeoJSON file
    :param file_paths: List of paths to the TIFF files
    :param output_base_dir: Directory to save the results
    :param cropland_values: List of pixel values considered as cropland
    :param threshold: Minimum number of years a pixel must be classified as cropland to be considered cropland
    """
    # Extract polygons and names of regions from the GeoJSON file
    regions = get_polygons_from_geojson(geojson_file)

    for region_name, polygon in regions:
        print(f"Processing region: {region_name}")

        # Set the output file path for each region
        output_file = os.path.join(output_base_dir, f"cropmap_{region_name}.tif")

        # Process the TIFF files, calculate cropland areas, and save the result
        process_cropmap(file_paths, polygon, cropland_values=cropland_values, threshold=threshold, output_file=output_file)

# Example execution
# List of file paths (cropmap files for 2013 to 2018)
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019]
file_paths = [f'/home/sehoon/Desktop/fallow_land/cropmap/Ukraine/masked_cropmap_{year}.tif' for year in years]

# Path to the GeoJSON file
geojson_file = "/home/sehoon/Desktop/fallow_land/Ukraine_Polygon_GAUL2015.geojson"

# Set the output directory
output_base_dir = "/mnt/raid5/1114/cropmap"

# Generate cropland maps for all regions
generate_cropmaps_by_region(
    geojson_file=geojson_file,
    file_paths=file_paths,
    output_base_dir=output_base_dir,
    cropland_values=[10, 11, 12, 20],  # Values considered as cropland
    threshold=5  # Consider as cropland if classified as cropland for at least 5 years
)